In [22]:
import pandas as pd
dt = pd.read_pickle('final_task_bad_data.pickle')

In [23]:
copy = dt.copy()
copy.reset_index(inplace=True)

In [24]:
# bad_close will contain all the "bad" rows
bad_close = copy[(copy["High"] < copy["Close"]) | (copy["Low"] > copy["Close"])]
bad_close

,Date,Open,High,Low,Close,Volume
150,2022-09-23 00:00:00-04:00,100.293137,100.851316,97.701578,9999.0,11976900.0
151,2022-09-26 00:00:00-04:00,98.628560,100.333012,97.741452,0.0,9760500.0
152,2022-09-27 00:00:00-04:00,99.206673,99.316317,95.119994,9999.0,13360200.0
300,2023-05-01 00:00:00-04:00,102.077323,102.276670,101.279918,-20.0,6909500.0


In [25]:
copy.loc[150:152, "Close"] = None
copy.loc[300, "Close"] = None
# recheck if there are more "bad" rows
bad_close = copy[(copy["High"] < copy["Close"]) | (copy["Low"] > copy["Close"])]
bad_close

,Date,Open,High,Low,Close,Volume


In [26]:
# bad_open will contain all the "bad" rows
bad_open = copy[(copy["High"] < copy["Open"]) | (copy["Low"] > copy["Open"])]
bad_open


,Date,Open,High,Low,Close,Volume
250,2023-02-16 00:00:00-05:00,97.200627,107.669094,105.356631,105.486214,9479900.0
296,2023-04-25 00:00:00-04:00,89.057157,99.386087,97.691612,97.691612,6511400.0


In [27]:
copy.loc[250, "Open"] = None
copy.loc[296, "Open"] = None
# recheck if there are more "bad" rows
bad_open = copy[(copy["High"] < copy["Open"]) | (copy["Low"] > copy["Open"])]
bad_open

,Date,Open,High,Low,Close,Volume


In [28]:
# check that all the "bad" rows have NaN value 
copy[copy.isna().any(axis=1)]

,Date,Open,High,Low,Close,Volume
10,2022-03-04 00:00:00-05:00,143.542169,143.791357,139.096666,140.262863,NaN
150,2022-09-23 00:00:00-04:00,100.293137,100.851316,97.701578,NaN,11976900.0
151,2022-09-26 00:00:00-04:00,98.628560,100.333012,97.741452,NaN,9760500.0
152,2022-09-27 00:00:00-04:00,99.206673,99.316317,95.119994,NaN,13360200.0
250,2023-02-16 00:00:00-05:00,NaN,107.669094,105.356631,105.486214,9479900.0
296,2023-04-25 00:00:00-04:00,NaN,99.386087,97.691612,97.691612,6511400.0
300,2023-05-01 00:00:00-04:00,102.077323,102.276670,101.279918,NaN,6909500.0


In [29]:
def mean_imputation_without_leakage(srs, high, low):
    filled_srs = srs.copy()
    for index in range(len(srs)):
        if pd.isnull(srs.iloc[index]):
            valid_values = srs.where((srs >= low[index]) & (srs <= high[index])).dropna() 
            filled_srs.iloc[index] = valid_values.expanding().mean().iloc[-1] 
    return filled_srs

In [30]:
# fix open and close
copy["Close"] = mean_imputation_without_leakage(copy['Close'], copy['High'], copy['Low'])
copy["Open"] = mean_imputation_without_leakage(copy['Open'], copy['High'], copy['Low'])

In [31]:
# recheck if there are still more "bad" rows
bad_close = copy[(copy["High"] < copy["Close"]) | (copy["Low"] > copy["Close"])]
bad_close

,Date,Open,High,Low,Close,Volume


In [32]:
# recheck if there are still more "bad" rows
bad_open = copy[(copy["High"] < copy["Open"]) | (copy["Low"] > copy["Open"])]
bad_open

,Date,Open,High,Low,Close,Volume


In [33]:
def mean_imputation_without_leakage(srs):
    return srs.fillna(srs.expanding().mean())

# fix Volume
copy["Volume"] = mean_imputation_without_leakage(copy['Volume'])

In [34]:
# check if there are still NaN values
copy[copy.isna().any(axis=1)]

,Date,Open,High,Low,Close,Volume


In [35]:
copy.to_pickle('fixed_data.pickle')

In [36]:
import wrds

conn=wrds.Connection()
tables = conn.list_tables(library='crsp')

Loading library list...
Done


In [37]:
def get_stock_data(symbol: str, start_date: str, end_date: str) -> pd.DataFrame:
    """
    Fetches stock data for a given symbol within a specified date range
    from WRDS CRSP database.

    Args:
        symbol (str): Ticker symbol of the stock.
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
        conn (wrds.Connection): Connection object to WRDS database.

    Returns:
        pd.DataFrame: DataFrame containing the fetched stock data.
    """
    query = f"""SELECT c.date AS Date,
                        c.permco AS company_code,
                        c.permno AS security_code,
                        c.openprc AS Open, 
                        c.askhi AS High,
                        c.bidlo AS Low, 
                        c.prc AS Close, 
                        c.vol AS Volume,
                        c.cfacpr AS splits_adjustment_factor,
                        c.ret AS adjusted_returns,
                        c.retx AS returns_without_dividends,
                        i.ticker AS symbol,
                        i.uesindustry AS industry,
                        d.divamt AS dividend_amount
                    FROM crsp.dsf c
                        JOIN crsp.stksecurityinfohdr i ON c.permno = i.permno
                        LEFT JOIN crsp.dse d ON c.permno = d.permno AND c.date = d.date
                    WHERE c.date >= '{start_date}'
                        AND c.date <= '{end_date}'
                        AND i.ticker = '{symbol}'
                """
    # conn = wrds.Connection()
    return conn.raw_sql(query, date_cols=['Date'])


df = get_stock_data(symbol='ANF', start_date='2018-01-01', end_date='2020-04-01')
df

,date,company_code,security_code,open,high,low,close,volume,splits_adjustment_factor,adjusted_returns,returns_without_dividends,symbol,industry,dividend_amount
0,2018-01-02,22200,83976,17.64,18.300,17.5100,18.20,2275956.0,1.0,0.044177,0.044177,ANF,CONDIS,NaN
1,2018-01-03,22200,83976,18.30,18.590,18.1300,18.53,2112677.0,1.0,0.018132,0.018132,ANF,CONDIS,NaN
2,2018-01-04,22200,83976,18.41,18.550,17.0400,18.20,3580819.0,1.0,-0.017809,-0.017809,ANF,CONDIS,NaN
3,2018-01-05,22200,83976,18.20,18.540,17.9900,18.28,2288929.0,1.0,0.004396,0.004396,ANF,CONDIS,NaN
4,2018-01-08,22200,83976,18.41,18.555,17.9507,18.31,2417879.0,1.0,0.001641,0.001641,ANF,CONDIS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,2020-03-26,22200,83976,9.71,10.620,9.1000,10.00,5052422.0,1.0,0.055966,0.055966,ANF,CONDIS,NaN
562,2020-03-27,22200,83976,9.40,10.100,9.0000,9.91,3034780.0,1.0,-0.009000,-0.009000,ANF,CONDIS,NaN
563,2020-03-30,22200,83976,9.80,9.800,9.1400,9.31,1588860.0,1.0,-0.060545,-0.060545,ANF,CONDIS,NaN
564,2020-03-31,22200,83976,9.31,9.585,8.7600,9.09,2346424.0,1.0,-0.023631,-0.023631,ANF,CONDIS,NaN


In [38]:
df["dividend_amount"] = df["dividend_amount"].fillna(0)
df["dir"] = (1 - (df["dividend_amount"] / df["close"].shift(1)))
df.insert(0, 'index', df.index)
df

,index,date,company_code,security_code,open,high,low,close,volume,splits_adjustment_factor,adjusted_returns,returns_without_dividends,symbol,industry,dividend_amount,dir
0,0,2018-01-02,22200,83976,17.64,18.300,17.5100,18.20,2275956.0,1.0,0.044177,0.044177,ANF,CONDIS,0.0,NaN
1,1,2018-01-03,22200,83976,18.30,18.590,18.1300,18.53,2112677.0,1.0,0.018132,0.018132,ANF,CONDIS,0.0,1.0
2,2,2018-01-04,22200,83976,18.41,18.550,17.0400,18.20,3580819.0,1.0,-0.017809,-0.017809,ANF,CONDIS,0.0,1.0
3,3,2018-01-05,22200,83976,18.20,18.540,17.9900,18.28,2288929.0,1.0,0.004396,0.004396,ANF,CONDIS,0.0,1.0
4,4,2018-01-08,22200,83976,18.41,18.555,17.9507,18.31,2417879.0,1.0,0.001641,0.001641,ANF,CONDIS,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,561,2020-03-26,22200,83976,9.71,10.620,9.1000,10.00,5052422.0,1.0,0.055966,0.055966,ANF,CONDIS,0.0,1.0
562,562,2020-03-27,22200,83976,9.40,10.100,9.0000,9.91,3034780.0,1.0,-0.009000,-0.009000,ANF,CONDIS,0.0,1.0
563,563,2020-03-30,22200,83976,9.80,9.800,9.1400,9.31,1588860.0,1.0,-0.060545,-0.060545,ANF,CONDIS,0.0,1.0
564,564,2020-03-31,22200,83976,9.31,9.585,8.7600,9.09,2346424.0,1.0,-0.023631,-0.023631,ANF,CONDIS,0.0,1.0


In [39]:
df['cdir'] = df['dir'].shift(-1)[::-1].cumprod()[::-1]
df['cdir'] = df['cdir'].fillna(1.0)
df

,index,date,company_code,security_code,open,high,low,close,volume,splits_adjustment_factor,adjusted_returns,returns_without_dividends,symbol,industry,dividend_amount,dir,cdir
0,0,2018-01-02,22200,83976,17.64,18.300,17.5100,18.20,2275956.0,1.0,0.044177,0.044177,ANF,CONDIS,0.0,NaN,0.908386
1,1,2018-01-03,22200,83976,18.30,18.590,18.1300,18.53,2112677.0,1.0,0.018132,0.018132,ANF,CONDIS,0.0,1.0,0.908386
2,2,2018-01-04,22200,83976,18.41,18.550,17.0400,18.20,3580819.0,1.0,-0.017809,-0.017809,ANF,CONDIS,0.0,1.0,0.908386
3,3,2018-01-05,22200,83976,18.20,18.540,17.9900,18.28,2288929.0,1.0,0.004396,0.004396,ANF,CONDIS,0.0,1.0,0.908386
4,4,2018-01-08,22200,83976,18.41,18.555,17.9507,18.31,2417879.0,1.0,0.001641,0.001641,ANF,CONDIS,0.0,1.0,0.908386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,561,2020-03-26,22200,83976,9.71,10.620,9.1000,10.00,5052422.0,1.0,0.055966,0.055966,ANF,CONDIS,0.0,1.0,1.000000
562,562,2020-03-27,22200,83976,9.40,10.100,9.0000,9.91,3034780.0,1.0,-0.009000,-0.009000,ANF,CONDIS,0.0,1.0,1.000000
563,563,2020-03-30,22200,83976,9.80,9.800,9.1400,9.31,1588860.0,1.0,-0.060545,-0.060545,ANF,CONDIS,0.0,1.0,1.000000
564,564,2020-03-31,22200,83976,9.31,9.585,8.7600,9.09,2346424.0,1.0,-0.023631,-0.023631,ANF,CONDIS,0.0,1.0,1.000000


In [40]:
df["adjusted_close"] = df["close"] * df["cdir"] / df["splits_adjustment_factor"]
df

,index,date,company_code,security_code,open,high,low,close,volume,splits_adjustment_factor,adjusted_returns,returns_without_dividends,symbol,industry,dividend_amount,dir,cdir,adjusted_close
0,0,2018-01-02,22200,83976,17.64,18.300,17.5100,18.20,2275956.0,1.0,0.044177,0.044177,ANF,CONDIS,0.0,NaN,0.908386,16.532627
1,1,2018-01-03,22200,83976,18.30,18.590,18.1300,18.53,2112677.0,1.0,0.018132,0.018132,ANF,CONDIS,0.0,1.0,0.908386,16.832394
2,2,2018-01-04,22200,83976,18.41,18.550,17.0400,18.20,3580819.0,1.0,-0.017809,-0.017809,ANF,CONDIS,0.0,1.0,0.908386,16.532627
3,3,2018-01-05,22200,83976,18.20,18.540,17.9900,18.28,2288929.0,1.0,0.004396,0.004396,ANF,CONDIS,0.0,1.0,0.908386,16.605298
4,4,2018-01-08,22200,83976,18.41,18.555,17.9507,18.31,2417879.0,1.0,0.001641,0.001641,ANF,CONDIS,0.0,1.0,0.908386,16.632549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,561,2020-03-26,22200,83976,9.71,10.620,9.1000,10.00,5052422.0,1.0,0.055966,0.055966,ANF,CONDIS,0.0,1.0,1.000000,10.000000
562,562,2020-03-27,22200,83976,9.40,10.100,9.0000,9.91,3034780.0,1.0,-0.009000,-0.009000,ANF,CONDIS,0.0,1.0,1.000000,9.910000
563,563,2020-03-30,22200,83976,9.80,9.800,9.1400,9.31,1588860.0,1.0,-0.060545,-0.060545,ANF,CONDIS,0.0,1.0,1.000000,9.310000
564,564,2020-03-31,22200,83976,9.31,9.585,8.7600,9.09,2346424.0,1.0,-0.023631,-0.023631,ANF,CONDIS,0.0,1.0,1.000000,9.090000


In [41]:
df["ratio"] = df["adjusted_close"] / df["close"]
df["adjusted_open"] = df["open"] * df["ratio"]
df["adjusted_high"] = df["high"] * df["ratio"]
df["adjusted_low"] = df["low"] * df["ratio"]
df

,index,date,company_code,security_code,open,high,low,close,volume,splits_adjustment_factor,...,symbol,industry,dividend_amount,dir,cdir,adjusted_close,ratio,adjusted_open,adjusted_high,adjusted_low
0,0,2018-01-02,22200,83976,17.64,18.300,17.5100,18.20,2275956.0,1.0,...,ANF,CONDIS,0.0,NaN,0.908386,16.532627,0.908386,16.023931,16.623465,15.905840
1,1,2018-01-03,22200,83976,18.30,18.590,18.1300,18.53,2112677.0,1.0,...,ANF,CONDIS,0.0,1.0,0.908386,16.832394,0.908386,16.623465,16.886897,16.469040
2,2,2018-01-04,22200,83976,18.41,18.550,17.0400,18.20,3580819.0,1.0,...,ANF,CONDIS,0.0,1.0,0.908386,16.532627,0.908386,16.723388,16.850562,15.478899
3,3,2018-01-05,22200,83976,18.20,18.540,17.9900,18.28,2288929.0,1.0,...,ANF,CONDIS,0.0,1.0,0.908386,16.605298,0.908386,16.532627,16.841478,16.341866
4,4,2018-01-08,22200,83976,18.41,18.555,17.9507,18.31,2417879.0,1.0,...,ANF,CONDIS,0.0,1.0,0.908386,16.632549,0.908386,16.723388,16.855104,16.306166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,561,2020-03-26,22200,83976,9.71,10.620,9.1000,10.00,5052422.0,1.0,...,ANF,CONDIS,0.0,1.0,1.000000,10.000000,1.000000,9.710000,10.620000,9.100000
562,562,2020-03-27,22200,83976,9.40,10.100,9.0000,9.91,3034780.0,1.0,...,ANF,CONDIS,0.0,1.0,1.000000,9.910000,1.000000,9.400000,10.100000,9.000000
563,563,2020-03-30,22200,83976,9.80,9.800,9.1400,9.31,1588860.0,1.0,...,ANF,CONDIS,0.0,1.0,1.000000,9.310000,1.000000,9.800000,9.800000,9.140000
564,564,2020-03-31,22200,83976,9.31,9.585,8.7600,9.09,2346424.0,1.0,...,ANF,CONDIS,0.0,1.0,1.000000,9.090000,1.000000,9.310000,9.585000,8.760000
